![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)
# Hybrid search with Redis

Hybrid search is all about combining lexical search with semantic vector search. This notebook will cover implementing 3 different hybrid search strategies with Redis:

- Linear combination of scores from lexical search (BM25) and vector search (Cosine Distance) with the aggregation API
- Client-Side Reciprocal Rank Fusion (RRF)
- Client-Side Reranking with a cross encoder model

Additional work is planed within the Redis core and ecosystem to add more flexible hybrid search capabilities in the future.

## Let's Begin!
<a href="https://colab.research.google.com/github/redis-developer/redis-ai-resources/blob/main/python-recipes/vector-search/02_hybrid_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### Install Packages

In [2]:
# NBVAL_SKIP
%pip install -q redis "redisvl>=0.3.4" numpy sentence-transformers

Note: you may need to restart the kernel to use updated packages.


### Data/Index Preparation
 
In this section, we will prepare the data necessary for our hybrid search implementations by loading a collection of **movie** objects. Each movie object will contain the following attributes: `title`, `rating`, `description`, and `genre`. 
 
To enhance our search capabilities, we will generate vector embeddings from the movie descriptions. This allows users to perform searches that not only rely on exact matches but also on semantic relevance, helping them find movies that align closely with their interests.

After preparing the data, we will populate a search index with these movie records, enabling efficient querying based on both lexical and vector-based search techniques.

**Note:** If you are running this notebook in a local environment, you may find that this next step is not necessary, as the data might already be available.

In [ ]:
# NBVAL_SKIP
!git clone https://github.com/redis-developer/redis-ai-resources.git temp_repo
!mv temp_repo/python-recipes/vector-search/resources .
!rm -rf temp_repo

### Install Redis Stack

Later in this tutorial, Redis will be used to store, index, and query vector
embeddings and full text fields. **We need to make sure we have a Redis
instance available.**

#### For Colab
Use the shell script below to download, extract, and install [Redis Stack](https://redis.io/docs/getting-started/install-stack/) directly from the Redis package archive.

In [ ]:
# NBVAL_SKIP
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

#### For Alternative Environments
There are many ways to get the necessary redis-stack instance running
1. On cloud, deploy a [FREE instance of Redis in the cloud](https://redis.com/try-free/). Or, if you have your
own version of Redis Enterprise running, that works too!
2. Per OS, [see the docs](https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/)
3. With docker: `docker run -d --name redis-stack-server -p 6379:6379 redis/redis-stack-server:latest`

### Define the Redis Connection URL

By default this notebook connects to the local instance of Redis Stack. **If you have your own Redis Enterprise instance** - replace REDIS_PASSWORD, REDIS_HOST and REDIS_PORT values with your own.

In [1]:
import os

# Replace values below with your own if using Redis Cloud instance
REDIS_HOST = os.getenv("REDIS_HOST", "localhost") # ex: "redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
REDIS_PORT = os.getenv("REDIS_PORT", "6379")      # ex: 18374
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")  # ex: "1TNxTEdYRDgIDKM2gDfasupCADXXXX"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"

### Create redis client, load data, generate embeddings

In [2]:
from redis import Redis

client = Redis.from_url(REDIS_URL)

In [3]:
import json

with open("resources/movies.json", 'r') as file:
    movies = json.load(file)

In [8]:
from redisvl.utils.vectorize import HFTextVectorizer

# load model for embedding our movie descriptions
model = HFTextVectorizer('sentence-transformers/all-MiniLM-L6-v2')

In [9]:
movie_data = [
    {
        **movie,
        "description_vector": model.embed(movie["description"], as_buffer=True, dtype="float32")
    } for movie in movies
]

In [11]:
len(movie_data)

20

In [12]:
movie_data[0]

{'title': 'Explosive Pursuit',
 'genre': 'action',
 'rating': 7,
 'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'description_vector': b'\x91f|=\xb6`\n;g\x92\xb7;3\xcb~\xbd\x16e\xce\xbb\xd7\x16J=P\xa7?=\xc8v\x95<i\xfa\x06\xbe\x12Y\xcf=4\x07p=D\xdb\r\xbd\x8d\xf2H\xbdfe\xc6<G\xdfa=t8\x16\xbc\xd4\xd3\x13<A\xaa\x1c=\x06\xef\x89<\xb6\xb0-<\x99\xb2\x9f\xbcZ\x0b\xc3\xbd\xa5NR=Zl\xf7\xbcN>\x17\xbe\x02\x1a\x05\xb9@u\xbf<\xd6\xe2b\xba\xd0\xa6\xa8\xbdo\xdc\xec\xbcQc%=N\xe7r\xbb\x1dOG==(\x85=y@\xa2\xbc7Z\xd0\xbdB%K\xbd\xba\xed\x94\xbcU\xddH=\xbe&F<\xbc*\xec<\x8c\xd8\x8d\xbd\xf3Z\x98<\x15\xa3\xa3=3g3\xbd$\xcd\xbd\xbd\xf7$\xf7;\xf6\xf4z=\x02\xb5\x8c=\x8d\x0e\xc6\xbdhI\x90\xbdq\x16\xbd;u\xe7\x0c\xbd&3\xc9\xbc\x82\xf8\xbb\xbc\xa7&u\xbb-\x8f\xca<\xf2\x7fJ=\x14\xaf*=\x87OU\xbd\xde\xf0\x95\xbc \x02\x19=\x1b\xf4K<\xd0\xc2\t=F\x83\xac=\x9e\xd7\xb8\xbd\xf3\xb5\x9c\xbdB\x85\x18=\xa4d&=\'3\xf8<\xd3\xf7\x88<Tv\xf2\xbb1=[\xbda\xac\xee\xbb4:A\x

## Define Redis index schema

In [16]:
from redisvl.schema import IndexSchema
from redisvl.index import SearchIndex

index_name = "movies"

schema = IndexSchema.from_dict({
  "index": {
    "name": index_name,
    "prefix": "movie",
  },
  "fields": [
    { "name": "title", "type": "text" },
    { "name": "description", "type": "text" },
    { "name": "genre", "type": "tag", "attrs": {"sortable": True}},
    { "name": "rating", "type": "numeric", "attrs": {"sortable": True}},
    {
        "name": "description_vector",
        "type": "vector",
        "attrs": {
            "dims": 384,
            "distance_metric": "cosine",
            "algorithm": "hnsw",
            "datatype": "float32"
        }
    }
  ]
})


index = SearchIndex(schema, client)
index.create(overwrite=True, drop=True)

16:55:38 redisvl.index.index INFO   Index already exists, overwriting.


### Populate index

In [18]:
index.load(movie_data)

['movie:af71c79937e04e8faf517e6aa7ac345c',
 'movie:2fb07b0d100e4d7c8e7438cc0814382f',
 'movie:e19b7d53fe334dde88e1e5cabaffe4d3',
 'movie:476abf9f05864993a7e87795f10b4ecf',
 'movie:3b682518ec21458f9852c04599e85ec9',
 'movie:0c16db53b7104858b04735fda785f4cd',
 'movie:a869fae2f3574e6a9374be0e2f52f7e7',
 'movie:c493e0edcf48449cac6e8f06c22f7306',
 'movie:457db737503d441c8e158b3b8a99ed76',
 'movie:9e4c3559c93e4dd4956b5c4caa198631',
 'movie:db9a74588cee4961a297a7a85f74c875',
 'movie:9da26bb36a214bf4b76da6676888fbf3',
 'movie:1f2c8ed55c944aa58812f969bfb5bb33',
 'movie:d5f356ce35174ce6a7134e3d07d11346',
 'movie:3e7184e070574a1187fb99cb2e4d09e4',
 'movie:09c1908809fd42ab9ffc1bbf6b50fcf6',
 'movie:94954dd933f34fd4bcd66acb03ae89e2',
 'movie:a26a14ef194249988c42b99cfc22ea62',
 'movie:b6e9967459c74ff4898c0990a0cd5769',
 'movie:f3981cdf54a044c1b5951e7a3fc56235']

# Hybrid Search Approaches

## 1. Linear Combination using Aggregation API
Now that our index is populated and ready, let's execute our first hybrid search technique. This query will calculate a linear combination of the BM25 score for our provided text search and the cosine distance between vectors calculated via a KNN vector query.

This style of query is useful when you want to combine the power of key words (BM25) with the flexibility of vector search.

First, let's set up what we need for the hybrid search including the user query and query vector.

In [131]:
user_query = "action adventure movie with great fighting scenes, crime busting, superheroes, and magic"
embedded_user_query = model.embed(user_query, as_buffer=True, dtype="float32")

In [214]:
def convert_user_query(user_query: str) -> str:
    """Convert a raw user query to a redis full text query joined by ORs"""
    return " | ".join([token.strip().strip(",").lower() for token in user_query.split()])

Next, we build a base `VectorQuery` that contains a KNN-style vector search.

In [195]:
from redisvl.query import VectorQuery, VectorRangeQuery, FilterQuery
from redisvl.query.filter import Text
from redisvl.redis.utils import convert_bytes, make_dict

query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="description_vector",
    num_results=3,
    return_fields=["title", "description"]
)

In [196]:
# Standard KNN vector search
index.query(query)

[{'id': 'movie:09c1908809fd42ab9ffc1bbf6b50fcf6',
  'vector_distance': '0.643690466881',
  'title': 'The Incredibles',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy."},
 {'id': 'movie:af71c79937e04e8faf517e6aa7ac345c',
  'vector_distance': '0.668439388275',
  'title': 'Explosive Pursuit',
  'description': 'A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.'},
 {'id': 'movie:0c16db53b7104858b04735fda785f4cd',
  'vector_distance': '0.698122441769',
  'title': 'Mad Max: Fury Road',
  'description': "In a post-a

In [215]:
str(Text("description") % convert_user_query(user_query))

'@description:(action | adventure | movie | with | great | fighting | scenes | crime | busting | superheroes | and | magic)'

In [198]:
# Add full-text portion
query.set_filter(Text("description") % convert_user_query(user_query))
query.query_string()

'@description:(action | adventure | movie | with | great | fighting | scenes | crime | busting | superheroes | and | magic)=>[KNN 3 @description_vector $vector AS vector_distance]'

This query string combines both a full-text search and a vector (KNN) search and will be passed to the aggregation API to combine using a simple linear combination (with `alpha` weight) and sort by `hybrid_score`.

In [216]:
from typing import Any, Dict, List, Optional
from redis.commands.search.aggregation import AggregateRequest, Desc


# TODO: scorer not in redis-py yet for aggregations
# This is in a PR for the library and should be available soon

class AggregateRequestWithScorer(AggregateRequest):
    _scorer: str = None

    def scorer(self, scorer: str):
        self._scorer = scorer
        return self

    def build_args(self):
        # @foo:bar ...
        ret = [self._query]

        if self._scorer:
            ret.extend(["SCORER", self._scorer])

        if self._with_schema:
            ret.append("WITHSCHEMA")

        if self._verbatim:
            ret.append("VERBATIM")

        if self._add_scores:
            ret.append("ADDSCORES")

        if self._cursor:
            ret += self._cursor

        if self._loadall:
            ret.append("LOAD")
            ret.append("*")
        elif self._loadfields:
            ret.append("LOAD")
            ret.append(str(len(self._loadfields)))
            ret.extend(self._loadfields)

        if self._dialect:
            ret.extend(["DIALECT", self._dialect])

        ret.extend(self._aggregateplan)

        return ret

Notes on aggregate query syntax 
- `.scorer`: specifies the scoring function to use either BM25 or TFIDF
- `.add_scores`: adds the scores to the result
- `.apply`: algebraic operations that can be customized for your use case
- `.load`: specifies fields to return - all in this case.
- `.sort_by`: sort the output based on the hybrid score and yield top 5 results
- `.dialect`: specifies the query dialect to use.

In [219]:
def linear_combo(query: str, alpha: float, distance_threshold: float = None) -> List[Dict[str, Any]]:
    req = (
        AggregateRequestWithScorer(query)
            .scorer("BM25")
            .add_scores()
            .apply(cosine_similarity="(2 - @vector_distance)/2", bm25_score="@__score")
            .apply(hybrid_score=f"{1-alpha}*@bm25_score + {alpha}*@cosine_similarity")
            .load("title", "description", "cosine_similarity", "bm25_score", "hybrid_score")
            .sort_by(Desc("@hybrid_score"), max=5)
            .dialect(4)
    )
    params = {'vector': embedded_user_query}

    if distance_threshold:
        params["distance_threshold"] = distance_threshold

    res = index.client.ft(index.schema.index.name).aggregate(req, query_params=params)

    # Perform output parsing
    if res:
        return [make_dict(row) for row in convert_bytes(res.rows)]

In [220]:
# Test it out
linear_combo(query.query_string(), alpha=0.7) # 70% of the hybrid search score is based on cosine similarity

[{'vector_distance': '0.643690466881',
  '__score': '0.968066079387',
  'title': 'The Incredibles',
  'description': "A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy.",
  'cosine_similarity': '0.67815476656',
  'bm25_score': '0.968066079387',
  'hybrid_score': '0.765128160408'},
 {'vector_distance': '0.824123203754',
  '__score': '0.284697498389',
  'title': 'Finding Nemo',
  'description': 'After his son is captured in the Great Barrier Reef and taken to Sydney, a timid clownfish sets out on a journey to bring him home.',
  'cosine_similarity': '0.587938398123',
  'bm25_s

## 2. Client-side fusion

Instead of relying on document scores like cosine similarity and BM25/TFIDF, we can fetch items and focus on their rank. This rank can be utilized to create a new ranking metric known as Reciprocal Rank Fusion (RRF). Although Redis does not currently support RRF natively, we can easily implement it on the client side.

In [221]:
def weighted_rrf(*ranked_lists, weights=None, k=60):
    """
    Perform Weighted Reciprocal Rank Fusion on N number of ordered lists.
    """
    item_scores = {}
    
    if weights is None:
        weights = [1.0] * len(ranked_lists)
    else:
        assert len(weights) == len(ranked_lists), "Number of weights must match number of ranked lists"
        assert all(0 <= w <= 1 for w in weights), "Weights must be between 0 and 1"
    
    for ranked_list, weight in zip(ranked_lists, weights):
        for rank, item in enumerate(ranked_list, start=1):
            if item not in item_scores:
                item_scores[item] = 0
            item_scores[item] += weight * (1 / (rank + k))
    
    # Sort items by their weighted RRF scores in descending order
    fused_list = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Return only the items, not their scores
    return [item for item, score in fused_list]

In [222]:
# Below is a simple example of RRF over a few lists of numbers
weighted_rrf([1, 2, 3], [2, 4, 6, 7, 8], [5, 6, 1, 2])

[2, 1, 6, 5, 4, 3, 7, 8]

In [223]:
vector_query = VectorQuery(
    vector=embedded_user_query,
    vector_field_name="description_vector",
    num_results=20,
    return_fields=["title", "description"],
    dialect=4
)

full_text_query = FilterQuery(
    filter_expression=(Text("description") % " | ".join([token.strip().strip(",").lower() for token in user_query.split()])),
    num_results=20,
    dialect=4,
    return_fields=["title", "description"]
).scorer("BM25").with_scores()

# Add optional "~" operator to front part of the full text query to expand the scope of potential results
full_text_query._query_string = "(~@description:(action | adventure | movie | with | great | fighting | scenes | crime | busting | superheroes | and | magic))"

# Run queries
vector_query_results = index.query(vector_query)
full_text_query_results = index.query(full_text_query)

In [224]:
# Fetched top 20 results from each
len(vector_query_results) == len(full_text_query_results) == 20

True

In [225]:
# Let's take a look at the results given the following user query (as a reminder)
print(user_query, "...")

weighted_rrf(
    [movie["title"] for movie in vector_query_results],
    [movie["title"] for movie in full_text_query_results]
)

action adventure movie with great fighting scenes, crime busting, superheroes, and magic ...


['The Incredibles',
 'Explosive Pursuit',
 'Mad Max: Fury Road',
 'Finding Nemo',
 'Fast & Furious 9',
 'The Dark Knight',
 'Aladdin',
 'John Wick',
 'Inception',
 'Black Widow',
 'Toy Story',
 'Skyfall',
 'The Avengers',
 'Despicable Me',
 'Madagascar',
 'Shrek',
 'The Lego Movie',
 'Gladiator',
 'Monsters, Inc.',
 'The Princess Diaries']

But say we want to give more weight to the vector query results in this case to boost semantic similarities contribution to the final rank:

In [226]:
weighted_rrf(
    [movie["title"] for movie in vector_query_results],
    [movie["title"] for movie in full_text_query_results],
    weights=[0.7, 0.3]
)

['The Incredibles',
 'Explosive Pursuit',
 'Mad Max: Fury Road',
 'The Dark Knight',
 'Fast & Furious 9',
 'Inception',
 'Despicable Me',
 'Finding Nemo',
 'John Wick',
 'The Avengers',
 'Black Widow',
 'Aladdin',
 'Skyfall',
 'Madagascar',
 'Toy Story',
 'Shrek',
 'The Lego Movie',
 'Gladiator',
 'Monsters, Inc.',
 'The Princess Diaries']

## 3. Client-side reranking

An alternative approach to RRF is to simply use an external reranker to order the final recommendations. RedisVL has built-in integrations to a few popular reranking modules.

In [240]:
from redisvl.utils.rerank import HFCrossEncoderReranker

# Load the ms marco MiniLM cross encoder model from huggingface
reranker = HFCrossEncoderReranker("cross-encoder/ms-marco-MiniLM-L-6-v2")

# Assemble the set of potentiel movie candidates from search results
movie_candidates = set(
    [f"Title: {movie['title']}. Description: {movie['description']}" for movie in vector_query_results] +
    [f"Title: {movie['title']}. Description: {movie['description']}" for movie in full_text_query_results]
)
movie_candidates

{'Title: Aladdin. Description: A kind-hearted street urchin and a power-hungry Grand Vizier vie for a magic lamp that has the power to make their deepest wishes come true.',
 'Title: Black Widow. Description: Natasha Romanoff confronts her dark past and family ties as she battles a new enemy.',
 'Title: Despicable Me. Description: When a criminal mastermind uses a trio of orphan girls as pawns for a grand scheme, he finds their love is profoundly changing him for the better.',
 'Title: Explosive Pursuit. Description: A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.',
 'Title: Fast & Furious 9. Description: Dom and his crew face off against a high-tech enemy with advanced weapons and technology.',
 'Title: Finding Nemo. Description: After his son is captured in the Great Barrier Reef and taken to Sydney, a timid clownfish sets out on a journey to bring him home.',
 'Title: Gladiator. Description: A betrayed Roman general seeks revenge agai

In [249]:
# Let the reranker do it's thing
def rerank(query: str, candidates: set, limit: int = 5) -> list:
    """Rerank the candidates based on the user query with an external model/module."""
    return reranker.rank(
        query=query,
        docs=[candidate for candidate in candidates],
        limit=limit,
        return_score=False
    )

In [248]:
rerank(user_query, movie_candidates)

[{'content': "Title: The Incredibles. Description: A family of undercover superheroes, while trying to live the quiet suburban life, are forced into action to save the world. Bob Parr (Mr. Incredible) and his wife Helen (Elastigirl) were among the world's greatest crime fighters, but now they must assume civilian identities and retreat to the suburbs to live a 'normal' life with their three children. However, the family's desire to help the world pulls them back into action when they face a new and dangerous enemy."},
 {'content': 'Title: The Dark Knight. Description: Batman faces off against the Joker, a criminal mastermind who threatens to plunge Gotham into chaos.'},
 {'content': 'Title: Explosive Pursuit. Description: A daring cop chases a notorious criminal across the city in a high-stakes game of cat and mouse.'},
 {'content': "Title: The Avengers. Description: Earth's mightiest heroes come together to stop an alien invasion that threatens the entire planet."},
 {'content': "Titl

# Wrap up
That's a wrap! Hopefully from this you were able to learn:
- How to implement simple vector search queries in Redis
- How to implement vector search queries with full-text filters
- How to implement hybrid search queries using the Redis aggregation API
- How to perform client-side fusion and reranking techniques